In [1]:
# Import and Install Necessary Packages
%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas

import pandas as pd
from eep153_tools.sheets import read_sheets
import re
from scipy.optimize import linprog as lp
import numpy as np

  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl.metadata (363 bytes)
Using cached eep153_tools-0.12.4-py2.py3-none-any.whl (4.9 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached python_gnupg-0.5.4-py2.py3-none-any.whl.metadata (2.0 kB)
Using cached python_gnupg-0.5.4-py2.py3-none-any.whl (21 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached gspread_pandas-3.3.0-py2.py3-none-any.whl.metadata (10 kB)
Using cached gspread_pandas-3.3.0-py2.py3-none-any.whl (27 kB)
  Attempting uninstall: gspread_pandas
    Found existing installation: gspread-pandas 2.2.3
    Uninstalling gspread-pandas-2.2.3:
      Successfully uninstalled gspread-pandas-2.2.3
Note: you may need to restart the kernel to use updated packages.


### Helper Function for Formatting IDs

This code defines a helper function `format_id` which takes an ID and an optional zero-padding parameter. It returns a formatted string version of the ID if possible. The function handles cases where the ID might be null, empty, or in a non-standard format. The code also sets a data URL for reference.

In [2]:
# Helper function to format an ID with optional zero-padding
def format_id(id, zeropadding=0):
    # Check if the id is null or an empty string or a dot, return None in such cases
    if pd.isnull(id) or id in ['', '.']:
        return None

    try:
        # Try converting the id to an integer and format it with zero-padding
        return ('%d' % id).zfill(zeropadding)
    except TypeError:
        # If a TypeError occurs, split the id by a period and take the first part, then remove any extra spaces and pad it
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        # If a ValueError occurs during conversion, return None
        return None

# URL to the data source (Google Spreadsheet)
data_url = "https://docs.google.com/spreadsheets/d/1z7hB1hWocUePYeoBpvR0_UW3LdX9MV82IwzpClkmsr4/edit?gid=1410082681#gid=1410082681"


### Load and Clean Data

This code loads the original recipes data from a Google Sheet, applies formatting to specific columns using the `format_id` helper function, and renames one of the columns for clarity.


In [3]:
# Load the original recipes data from the specified Google Sheet (sheet named "recipes")
og_recipes = read_sheets(data_url, sheet="recipes")

# Clean and transform the data:
# - Apply the format_id function to 'parent_foodcode' and 'ingred_code' columns to standardize their format
# - Rename the 'parent_desc' column to 'recipe' for better clarity
og_recipes = (og_recipes
              .assign(
                  parent_foodcode=lambda df: df["parent_foodcode"].apply(format_id),
                  ingred_code=lambda df: df["ingred_code"].apply(format_id)
              )
              .rename(columns={"parent_desc": "recipe"}))

In [37]:
# Define a list of key food items to INCLUDE in ingredient descriptions
key_foods = [
    "Yogurt, Greek", "Cheese, Cottage", "lowfat, 1% Milk", "milk, lowfat" "Cheese, Parmesan", "Banana", "Apple", "Orange",
    "Avocado", "figs", "dates", "raisins", "apricots, dried", "Grapefruit", "Grapes",
    "pear", "Peach", "watermelon", "Oats", "Bread, rye", "Brown Rice", "Pasta",
    "Quinoa", "Rolled Oats", "Rice Cakes", "Whole Grain Cereal", "Special K", "Bread, whole-wheat", 
    "Bread, whole-mutligrain", "Popcorn, Air-popped", "Millet",
    "whole grain pasta", "Almonds", "Peanut Butter", "Chicken", "Egg", "Tofu",
    "Lentils", "Beans, Black", "Tuna", "Salmon", "Soup, Bean ",
    "Steak", "Tilapia", "Pork", "Venison", "Cod", "Ground turkey", "turkey, Ground",
    "beef, ground", "Ground beef", "Tempeh", "chickpea", "beans, kidney", "Sweet Potato", "Potato",
    "Spinach", "Broccoli", "Bell Pepper", "Carrot", "Beets", "peas", "Tomato",
    "Creatine", "Omega-3", "BCAAs", "Blueberries", "Strawberries", "juice, raw",
    "Garlic", "Lemon", "Onion", "Asparagus", "kale", "collards", "chard, swiss", "protein powder", "brussel sprouts", "oat", "avacado",
    "sunflower", "salmon", "tuna", "mackerel", "fish"
    
]

# Define a list of foods or terms to EXCLUDE
key_excludable = [
    "sugar", "syrup", "soda", "candy", "artificial", "processed", "preservative",
    "yolk", "Fruit juice", "juice drink", "Sunny D", "sweetened", "added sugar", "liver", "babyfood", "baby food"
]

# Escape the items so that parentheses and other special characters are treated literally
escaped_key_foods = [re.escape(food) for food in key_foods]
escaped_excludable = [re.escape(term) for term in key_excludable]

# Wrap each escaped term in a non-capturing group '(?: ... )' before joining with '|'
include_pattern = '|'.join(f"(?:{term})" for term in escaped_key_foods)
exclude_pattern = '|'.join(f"(?:{term})" for term in escaped_excludable)

# 1) Include mask: meals that have at least one ingredient containing a key food
meal_mask_include = og_recipes.groupby('parent_foodcode')['ingred_desc'] \
    .transform(lambda x: x.str.contains(include_pattern, case=False, na=False).any())

# 2a) Exclude mask for INGREDIENTS: meals that have any ingredient containing an excludable term
meal_mask_exclude_ingredients = og_recipes.groupby('parent_foodcode')['ingred_desc'] \
    .transform(lambda x: x.str.contains(exclude_pattern, case=False, na=False).any())

# 2b) Exclude mask for RECIPE NAMES: meals whose recipe name contains an excludable term
meal_mask_exclude_names = og_recipes.groupby('parent_foodcode')['recipe'] \
    .transform(lambda x: x.str.contains(exclude_pattern, case=False, na=False).any())

# Combine both ingredient and recipe-name exclusions
meal_mask_exclude_total = meal_mask_exclude_ingredients | meal_mask_exclude_names

# 3) Final mask: include meals that pass the "include" filter AND do not match the exclusion filter
final_mask = meal_mask_include & (~meal_mask_exclude_total)

# Filter the original recipes dataset
recipes = og_recipes[final_mask]

In [38]:
# Load nutrition data and merge
nutrition = read_sheets(data_url, sheet="nutrients") \
            .assign(ingred_code=lambda df: df["ingred_code"].apply(format_id))

### Process and Aggregate Nutrient Information

This section makes a copy of the filtered recipes, normalizes ingredient weights to percentages, and merges nutrient information. Then, it scales nutrient values by their ingredient's normalized weight and aggregates the nutrient profile by meal. Finally, the code extracts recipe names for further use.

In [39]:
# Make an explicit copy of recipes before modifying
recipes = recipes.copy()

# Normalize ingredient weights to percentages by dividing by the total weight per meal.
# Using .loc for assignment ensures we're modifying the DataFrame in place.
recipes.loc[:, 'ingred_wt'] = recipes['ingred_wt'] / recipes.groupby('parent_foodcode')['ingred_wt'].transform("sum")

# Merge nutrient information into recipes on the 'ingred_code' column.
# This performs a left join, ensuring all recipes are kept.
df = recipes.merge(nutrition, how="left", on="ingred_code")

# Identify numeric columns (e.g., nutrient values) in the merged DataFrame.
numeric_cols = list(df.select_dtypes(include=["number"]).columns)

# Remove 'ingred_wt' from the list as we don't want to scale it.
numeric_cols.remove("ingred_wt")

# Multiply each nutrient value by the normalized ingredient weight to get weighted nutrient values.
df[numeric_cols] = df[numeric_cols].mul(df["ingred_wt"], axis=0)

# Aggregate nutrient profiles by meal (identified by 'parent_foodcode').
# For nutrient columns, sum their weighted values; for the recipe name, take the first occurrence.
df = df.groupby('parent_foodcode').agg({
    **{col: "sum" for col in numeric_cols},
    "recipe": "first"
})

# Rename the index to 'recipe_id' for clarity.
df.index.name = "recipe_id"

# Extract recipe names for further use.
food_names = df["recipe"]

### Load Prices and Map to Food Names

This code loads pricing data from a Google Sheet, applies ID formatting, and filters prices for a specific year. It then matches the price data with the corresponding recipes based on common food codes, maps the prices to food names, and prepares a transposed version of the nutrient data for further analysis.


In [40]:
# Load prices data from the "prices" sheet, selecting only the necessary columns.
prices = read_sheets(data_url, sheet="prices")[["food_code", "year", "price"]]

# Format the 'food_code' column using the helper function 'format_id'
prices["food_code"] = prices["food_code"].apply(format_id)

# Set a multi-index using 'year' and 'food_code' for easier slicing and alignment.
prices = prices.set_index(["year", "food_code"])

# Filter the prices data to include only records for the year "2017/2018".
prices = prices.xs("2017/2018", level="year")

# Remove rows where the price is missing.
prices = prices.dropna(subset="price")

# Find the intersection of food codes that are common between our aggregated recipes (df) and the prices data.
common_recipes = df.index.intersection(prices.index)

# Subset both the recipes and prices data to only include common recipes.
df = df.loc[common_recipes]
prices = prices.loc[common_recipes]

# Map the index (food codes) in the prices data to food names using the previously extracted 'food_names' series.
prices.index = prices.index.map(food_names)

# Transpose the nutrient data for further analysis or processing.
A_all = df.T

In [41]:
# Load RDA data (nutrient constraints)
rda = read_sheets(data_url, sheet="rda")
rda = rda.set_index("Nutrient")

# Define the Diet Minimizer Function

This function, `diet_minimizer`, uses linear programming to optimize a daily diet based on nutrient constraints for a given sex and athlete type. It constructs nutrient constraints from recommended dietary allowances (RDA) and upper limits (UL), then minimizes cost while meeting these constraints.

In [42]:
def diet_minimizer(sex, athlete_type):
    import numpy as np  
    group = f"{sex}_{athlete_type}"
    
    # Create nutrient constraints based on the chosen demographic
    bmin = pd.to_numeric(rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group], errors='coerce')
    bmax = pd.to_numeric(rda.loc[rda['Constraint Type'].isin(['UL']), group], errors='coerce')

    # Remove non-finite values
    bmin = bmin[np.isfinite(bmin)]
    bmax = bmax[np.isfinite(bmax)]

    # Filter constraints to only include nutrients available in A_all.
    bmin = bmin[bmin.index.isin(A_all.index)]
    bmax = bmax[bmax.index.isin(A_all.index)]

    # Remove excluded foods from A_all and prices before optimization
    filtered_A_all = A_all.loc[~A_all.index.isin(key_excludable)]
    filtered_prices = prices.loc[~prices.index.isin(key_excludable)]

    # Ensure reindexing aligns with filtered food data
    Amin = filtered_A_all.reindex(bmin.index).dropna(how='all')
    Amax = filtered_A_all.reindex(bmax.index).dropna(how='all')

    # Combine constraints
    b = pd.concat([bmin, -bmax]).dropna()
    A = pd.concat([Amin, -Amax])

    # Convert to NumPy arrays
    b = b.to_numpy().flatten()  
    A = A.to_numpy()
    
    # Prepare cost vector (filtered)
    p = filtered_prices["price"].to_numpy()

    # Tolerance for negligible quantities
    tol = 1e-6

    # Import linear programming solver
    from scipy.optimize import linprog as lp

    # Check that b contains only finite values
    if not np.all(np.isfinite(b)):
        raise ValueError("The constraint vector b contains non-finite values!")

    # Solve the linear programming problem
    result = lp(p, -A, -b, method='highs')

    # Extract optimized diet quantities
    diet_quantities = pd.Series(result.x, index=filtered_prices.index)
    total_cost = result.fun  

    # Select foods with quantities above tolerance threshold
    selected_foods = diet_quantities[diet_quantities >= tol]

    # Create DataFrame listing foods and their cost per 100g
    df_foods = pd.DataFrame({
        "Food": selected_foods.index,
        "Cost per 100g": [float(filtered_prices.loc[food, 'price']) for food in selected_foods.index]
    })

    print(f"Your daily diet is ${total_cost:.2f}")
    return df_foods

In [45]:
male_endurance = diet_minimizer("Male", "Endurance")

Your daily diet is $3.61


In [46]:
diet_minimizer("Male", "Strength")

Your daily diet is $5.50


,Food,Cost per 100g
0,"Carp, steamed or poached",0.782609
1,"Mackerel, canned",0.594039
2,"Egg, whole, boiled or poached",0.452410
3,"Egg, whole, fried with oil",0.398344
4,"Kidney beans, from dried, fat added",0.183994
5,"Pasta, gluten free",0.114248
6,"Oatmeal, instant, plain, made with water, no a...",0.146437
7,"Beans and rice, with tomatoes",0.178637
8,"Potato chips, reduced fat, unsalted",1.053719
9,"Collards, NS as to form, cooked",0.370074


In [36]:
female_strength =diet_minimizer('Female', 'Strength')
female_strength

Your daily diet is $5.06


,Food,Cost per 100g
0,"Chitterlings, cooked",0.487983
1,"Mackerel, canned",0.594039
2,"Egg, whole, fried with oil",0.398344
3,"Kidney beans, from dried, fat added",0.183994
4,"Pasta, gluten free",0.114248
5,"Oatmeal, instant, plain, made with water, no a...",0.146437
6,"Beans and rice, with tomatoes",0.178637
7,"Potato chips, reduced fat, unsalted",1.053719
8,"Collards, NS as to form, cooked",0.370074


In [21]:
def add_custom_ingredients(diet_df, custom_food_servings):
    """
    Adds the cheapest meal(s) matching each custom food to the diet,
    scaling each added food's cost by its specified serving size.
    
    Parameters
    ----------
    diet_df : pd.DataFrame
        The current daily diet DataFrame (output of diet_minimizer),
        with columns ["Food", "Cost per 100g"].
    custom_food_servings : dict
        A dictionary mapping custom food strings to their desired serving sizes (in grams).
        Example: {"rice cake": 50, "Apple": 80}
        
    Returns
    -------
    pd.DataFrame
        Updated daily diet DataFrame with new rows added for each matched custom food.
        Each new row includes "Food", "Cost per 100g", "Serving (g)", and "Cost Contribution".
    """
    # Make a copy so we don't modify the original diet_df
    updated_diet = diet_df.copy()
    
    # Save the original cost from the provided diet_df (assumed to be already correctly scaled)
    original_cost = updated_diet["Cost per 100g"].sum()
    
    custom_rows = []
    
    for food, serving_size in custom_food_servings.items():
        # Find rows in the global 'recipes' DataFrame where the 'recipe' name contains the food string.
        mask = recipes['recipe'].str.contains(food, case=False, na=False)
        matching_meals = recipes[mask]['parent_foodcode'].unique()

        if len(matching_meals) == 0:
            print(f"No match found for '{food}' in recipe names.")
            continue

        cheapest_price = float('inf')
        cheapest_meal_name = None

        # Loop through each matching meal and find the cheapest option.
        for meal_code in matching_meals:
            # Skip if meal_code is not present in the aggregated nutrient DataFrame 'df'
            if meal_code not in df.index:
                continue

            meal_name = df.loc[meal_code, 'recipe']

            # Look up the price using the meal name. If multiple rows match, take the first one.
            meal_price_info = prices.loc[meal_name, 'price']
            if isinstance(meal_price_info, pd.Series):
                meal_price_info = meal_price_info.iloc[0]

            # Convert the price to a float (if needed)
            meal_cost = float(meal_price_info)

            if meal_cost < cheapest_price:
                cheapest_price = meal_cost
                cheapest_meal_name = meal_name

        if cheapest_meal_name is None:
            print(f"No priced meal found for '{food}' among matches.")
            continue

        # Calculate the serving fraction (serving_size as fraction of 100g)
        serving_fraction = serving_size / 100.0
        # Adjust the cost to reflect the actual serving size
        adjusted_cost = serving_fraction * cheapest_price

        # Create a new row for the custom food with its serving size and adjusted cost
        new_row = pd.DataFrame({
            "Food": [cheapest_meal_name],
            "Cost per 100g": [cheapest_price],
            "Serving (g)": [serving_size],
            "Cost Contribution": [adjusted_cost]
        })
        custom_rows.append(new_row)
    
    if custom_rows:
        custom_df = pd.concat(custom_rows, ignore_index=True)
        # Append the custom rows to the original diet
        updated_diet = pd.concat([updated_diet, custom_df], ignore_index=True)
    else:
        custom_df = pd.DataFrame()

    # Calculate custom cost separately: sum of "Cost Contribution" from custom_df
    custom_cost = custom_df["Cost Contribution"].sum() if not custom_df.empty else 0.0
    
    # Total cost is the sum of the original diet cost and the custom cost.
    total_cost = original_cost + custom_cost
    
    print(f"Updated daily diet total cost is: ${total_cost:.2f}")
    
    # Optionally, if you want to remove the extra columns before returning:
    updated_diet = updated_diet.drop(columns=['Serving (g)', 'Cost Contribution'], errors='ignore')
    
    return updated_diet

In [27]:
custom_food_servings = {
    "rice cake": 5,
    "Apple": 37
}

add_custom_ingredients(male_endurance, custom_food_servings)

Updated daily diet total cost is: $3.90


,Food,Cost per 100g
0,"Carp, baked or broiled, fat added",1.241814
1,"Egg, whole, fried with oil",0.398344
2,"Split peas, from dried, fat added",0.140336
3,"Peanut butter, lower sodium",0.513910
4,"Pasta, gluten free",0.114248
5,"Millet, no added fat",0.061534
6,Cereal (Kellogg's Special K),0.800496
7,"Orange juice, 100%, freshly squeezed",0.180944
8,"Potato, boiled, from fresh, peel eaten, made w...",0.236255
9,Rice cake,1.535776
